In [1]:
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pdb
from Patch_ana import patch_pip, spon_ana
import seaborn as sns
import pickle
from matplotlib.backends.backend_pdf import PdfPages
plt.style.use('classic')

% matplotlib inline
# % matplotlib notebook

%load_ext autoreload
% autoreload

In [2]:
post_2Hz_reci = pd.read_pickle('/data/ana_post_2Hz_reci')

In [3]:
post_2Hz_reci

,Before,After,File name,elimi,spon_elimi_bef
121,"{'resp1_region': [12502, 12503, 12504, 12505, ...","{'resp1_region': [12502, 12503, 12504, 12505, ...",2017081802,30,"[0, 1, 15, 20, 21, 22, 25, 32, 39, 42, 78]"
122,"{'resp1_region': [12502, 12503, 12504, 12505, ...","{'resp1_region': [12502, 12503, 12504, 12505, ...",2017082302,50,"[0, 3, 24, 25, 26, 37, 47, 50]"
123,"{'resp1_region': [12502, 12503, 12504, 12505, ...","{'resp1_region': [12502, 12503, 12504, 12505, ...",2017082304,40,"[16, 18, 19]"
124,"{'resp1_region': [12502, 12503, 12504, 12505, ...","{'resp1_region': [12502, 12503, 12504, 12505, ...",2017082402,40,"[0, 2, 4, 10, 35, 54]"
125,"{'resp1_region': [12502, 12503, 12504, 12505, ...","{'resp1_region': [12502, 12503, 12504, 12505, ...",2017083001,45,"[0, 9, 20, 22, 25, 31, 35, 49, 52, 56, 59]"
126,"{'resp1_region': [12502, 12503, 12504, 12505, ...","{'resp1_region': [12502, 12503, 12504, 12505, ...",2017083101,30,"[3, 5, 12, 13, 14, 15, 16, 19, 24, 44, 50, 67,..."


In [4]:
# spon_elimi_bef = post_2Hz_reco.spon_elimi_bef
# spon_elimi_aft = {}
# spon_elimi_bef[121] = [0, 1, 15, 20, 21, 22, 25, 32, 39, 42, 78]
# spon_elimi_bef[122] = [0, 3, 24, 25, 26, 37, 47, 50]
# spon_elimi_bef[123] = [16, 18, 19]
# spon_elimi_bef[124] = [0, 2, 4, 10, 35, 54]
# spon_elimi_bef[125] = [0, 9, 20, 22, 25, 31, 35, 49, 52, 56, 59]
# spon_elimi_bef[126] = [3, 5, 12, 13, 14, 15, 16, 19, 24, 44, 50, 67, 68, 70]

In [5]:
list(post_2Hz_reci.index)

[121, 122, 123, 124, 125, 126]

In [6]:
with open('/data/psc_test_{}'.format(choice), 'wb') as fp:
    pickle.dump(psc_dic, fp)

NameError: name 'choice' is not defined

In [ ]:
# post_2Hz_reci.index
for choice in post_2Hz_reci.index:
    spon_list = post_2Hz_reci.Before.loc[choice]['spon_trace'].copy()
    elimi_list = post_2Hz_reci.loc[choice]['spon_elimi_bef'].copy()
    spon_list = spon_ana.pop_Spon_Trace(spon_list, elimi_list)
    spon_trace = -1 * np.hstack(spon_list)
    
    plt.figure()
    plt.plot(spon_trace)
    
    # Obtain the template
    popt, xdata, template = spon_ana.template_Gen(post_2Hz_reci, index=choice, bef_aft=0)
    
    plt.figure()
    plt.plot(template)
    plt.plot(spon_ana.fun_exp(xdata, *popt), label='raw fit')
    plt.plot(spon_ana.fun_exp(xdata, *popt*2), label='scale up')
    plt.plot(spon_ana.fun_exp(xdata, *popt/2), label='scale down')
    plt.legend()
    
    psc_dic = {}

    for i in range(len(spon_list)):
        # detection locations of PSCs
        psc_loc_raw = spon_ana.spon_detect(spon_list[i], spon_ana.fun_exp(xdata, *popt))
        psc_loc_up = spon_ana.spon_detect(spon_list[i], spon_ana.fun_exp(xdata, *popt*2))
        psc_loc_down = spon_ana.spon_detect(spon_list[i], spon_ana.fun_exp(xdata, *popt/2))
        psc_list= list(set(np.concatenate([psc_loc_raw,psc_loc_up, psc_loc_down])))
        psc_list.sort()
        psc_loc = np.array(psc_list).reshape(1,-1)
        # Obtain amplitude, decay, onset time constant of each psc
        psc_dic[i] = spon_ana.psc_search(spon_list[i], psc_loc)
        
    with open('/data/psc_test_{}'.format(choice), 'wb') as fp:
        pickle.dump(psc_dic, fp)
    
    print('saving psc data from %d ...' % choice)
    
    for j in range(len(psc_dic)):
        for i in range(len(psc_dic[j])):
            plt.figure()
            plt.plot(psc_dic[j][i]['trace'])
            plt.title("%d th resp, %d th trace, decay=%0.5f, error=%0.5f  " % (j, i, psc_dic[j][i]['decay_tau'], np.sqrt(psc_dic[j][i]['fit_error'])*10**10))
            plt.plot(np.arange(len(psc_dic[j][i]['decay_fit']))+np.argmax(psc_dic[j][i]['trace']), psc_dic[j][i]['decay_fit'])

### 1. Create preliminary template through fitting response to stimulation

In [ ]:
with open('/data/psc_test_{}'.format(choice), 'rb') as fp:
    psc_dic = pickle.load(fp)

In [ ]:
amp = []
decay_tau = []
onset_tau = []
for j in range(len(psc_dic)):
    for i in range(len(psc_dic[j])):
        amp.append(psc_dic[j][i]['amp'])
        decay_tau.append(psc_dic[j][i]['decay_tau'])
        onset_tau.append(psc_dic[j][i]['onset_tau'])

In [ ]:
amp_data = np.array(amp)
decay_tau_data = np.array(decay_tau)
onset_tau_data = np.array(onset_tau)

In [ ]:
plt.plot(amp_data, decay_tau, 'o', alpha=0.3)
plt.ylim([0, 0.005])

In [ ]:
plt.hist(decay_tau, range=(0,0.005), bins=100)

In [ ]:
plt.plot(amp_data, onset_tau, 'o')

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(np.array(onset_tau), np.array(decay_tau), np.array(amp_data))
plt.show()

### 2. Extract examplary large and isolated events using a high detection threshold

### 3. Create a final template by fitting a function to the average of the exemplary events

### 4. Extract all events with the final template using a low detection criterion threshold

### 5. Eliminate false-positive, add false-negative events

In [ ]:
# Create